# Modular Gurobi Solver 

In [13]:
# %matplotlib inline
# import matplotlib.pyplot as plt
import numpy as np 
import pandas as pd
import networkx as nx
from parse import *
from utils  import *
# from pulp import *
from math import *
import gurobipy as gp
from gurobipy import GRB

## Solver

In [2]:
def parse_inputs(inputs, K):
    df = inputs
    df = pd.DataFrame([x.split() for x in df[0].tolist() ])
    df = df.loc[2:,:]
    df["i"] = df.loc[:,0]
    df["j"] = df.loc[:,1]
    df["sadness"] = df.loc[:,3]
    df["happiness"] = df.loc[:,2]
    df = df[["i", "j", "sadness", "happiness"]]

    i_id = [int(i) for i in list(df["i"])]
    j_id = [int(i) for i in list(df["j"])]
    sadness = [float(i) for i in list(df["sadness"])]*K
    happiness = [float(i) for i in list(df["happiness"])]*K

    sadness_1 = [float(i) for i in list(df["sadness"])]
    happiness_1 = [float(i) for i in list(df["happiness"])]
    
    return i_id, j_id, sadness, happiness, sadness_1, happiness_1 

In [3]:
def create_variables(N, K, dict_e, dict_v):
    # edges 
    edges_1 = list(dict_e.values())

    # Create another edge array with different sort
    edges_2 = [dict_e.get(i) for i in sorted(dict_e.keys(), key=lambda x: (x[2], x[4]))]

    # Create another edge array for constraint 4 
    edges_3 = [dict_e.get(i) for i in sorted(dict_e.keys(), key=lambda x: (x[2], x[6]))]


    # vertices 
    vertices_1 = list(dict_v.values())

    # Create another edge array with different sort
    vertices_2 = [dict_v.get(i) for i in sorted(dict_v.keys(), key=lambda x: x[2])]
    
    # Quadratic variables 1 
    v_list = np.array(np.array_split(vertices_1, K)).tolist()
    v_big = []
    for i in range(K):
        j = 0
        while j != N:
            for _ in range(j,N-1):
                v_big.append(v_list[i][j])
            j += 1

    # Quadratic variables 2
    v_list_2 = [i[1:] for i in v_list]
    v_big_2 = []
    for h in range(K):
        j = 0
        while j != N:
            for i in range(j,N-1):
                v_big_2.append(v_list_2[h][i])
            j += 1
            i = j
    
    return edges_1, edges_2, edges_3, vertices_1, vertices_2, v_big, v_big_2

In [4]:
def constraint_helper(edges_1, sadness_1, vertices_2, N, K):
    aa = []
    h = 0
    for i in range(K):
        for j in range(comb(N,2)):
            aa.append(edges_1[h] * sadness_1[j])
            h += 1
    constraint_1 = [sum(aa[i:i+ comb(N,2)]) for i in range(0, len(aa), comb(N,2))]
    constraint_2 = [sum(vertices_2[i:i + K]) for i in range(0, len(vertices_2), K)]
    return constraint_1, constraint_2

In [35]:
def gurobi_solver(path, rooms, time_limit):
    '''
    Main solver function 
    
    path = location of the input file 
    rooms = number of rooms to open 
    time_limit = time limit for model runtime
    '''
    # Parsing input data 
    inputs = pd.read_csv(path, header = None)
    N = int(list(inputs[0])[0])
    s_max = float(list(inputs[0])[1])
    K = rooms
    i_id, j_id, sadness, happiness, sadness_1, happiness_1 = parse_inputs(inputs, K)

    # Initialize model
    m = gp.Model("proj")
    m.setParam('OutputFlag', 0)
    m.setParam("TuneOutput", 0)

    # Create arrays to help with methods 
    dict_e = {}
    for k in range(K):
        for i in range(comb(N, 2)):
            dict_e["e_{0}_{1}_{2}".format(i_id[i],j_id[i],k)] = m.addVar(name= "e_{0}_{1}_{2}".format(i_id[i],j_id[i],k), vtype = GRB.BINARY)

    dict_v = {}
    for j in range(K):
        for i in range(N):
            dict_v["v_{0}_{1}".format(i,j)] = m.addVar(name = "v_{0}_{1}".format(i,j),  vtype = GRB.BINARY)


    edges_1, edges_2, edges_3, vertices_1, vertices_2, v_big, v_big_2 = create_variables(N, K, dict_e, dict_v)

    # Set Object function 
    m.setObjective(sum(edges_1[i] * happiness[i] for i in range(len(happiness))), GRB.MAXIMIZE)

    # Helper method to create constraints 
    constraint_1, constraint_2 = constraint_helper(edges_1, sadness_1, vertices_2, N, K)

    # Add constraints 
    # Constraint 1 
    for i in range(len(constraint_1)):
        m.addConstr(constraint_1[i] <=  s_max // K) # Changes from float division 
    # Constraint 3
    for i in range(len(constraint_2)):
        m.addConstr(constraint_2[i] == 1)
    # Constraint 3 
    for i in range(len(edges_1)):
        m.addConstr(edges_1[i] == v_big[i] * v_big_2[i])

    # Optimize model 
    m.setParam('TimeLimit', time_limit)
    m.optimize()

    # Get outputs 
    pairs = m.getVars()[0 : len(m.getVars()) - N*K]
    people = m.getVars()[len(m.getVars()) - N*K : len(m.getVars())]
    try:
        people_values = [v.X for v in people]
        people_values = np.array_split(people_values, K)
        pair_values = [v.X for v in pairs]
        output_dict = {}
        for k in range(K):
            arr = []
            for i in range(N):
                if people_values[k][i] == 1: 
                    arr += [i]
                    output_dict[k] = arr
    # Outputs 
        total_happiness = np.sum(np.multiply(pair_values, happiness))
        total_sadness = np.sum(np.multiply(pair_values, sadness))
#         print("Total Happiness =", total_happiness)
#         print("Total Sadness =", total_sadness)
        return (output_dict)
    except:
        print("No/Infeasible Solution with {0} rooms".format(rooms))

In [6]:
# gurobi_solver("phase1/test50.in", 17, 60*10)

In [7]:
for room in range(1, 10):
    gurobi_solver("phase1/test20.in", room, 20)

Using license file /Users/smeetpatel/gurobi.lic
Academic license - for non-commercial use only - expires 2021-01-22
No/Infeasible Solution with 1 rooms
No/Infeasible Solution with 2 rooms
No/Infeasible Solution with 3 rooms
No/Infeasible Solution with 4 rooms
Total Happiness = 210.74900000000002
{0: [2, 5, 10, 14], 1: [3, 4, 11, 17], 2: [9, 10, 14, 17], 3: [6, 8, 15, 18], 4: [0, 7, 14, 16]}
Total Happiness = 185.02100000000002
{0: [3, 14, 16], 1: [2, 7, 13], 2: [4, 10, 16, 18], 3: [6, 9, 17, 19], 4: [0, 8, 18], 5: [5, 13, 15]}
Total Happiness = 177.256
{0: [4, 9, 11, 17], 1: [6, 16, 19], 3: [0, 8, 10, 14], 4: [5, 13, 15], 5: [2, 7, 13], 6: [3, 11, 17]}
Total Happiness = 152.706
{0: [8, 15, 18], 2: [0, 10, 14], 3: [5, 10], 4: [2, 12, 13], 5: [6, 16, 19], 6: [7, 9, 19], 7: [3, 4, 17]}
Total Happiness = 151.625
{0: [8, 15, 18], 2: [2, 13, 15], 3: [6, 9, 19], 4: [5, 10], 5: [3, 4, 12], 6: [7, 13, 15], 8: [0, 10, 14]}


In [11]:
for room in range(1, 10):
    gurobi_solver("phase1/test10.in", room, 10)

No/Infeasible Solution with 1 rooms
Total Happiness = 72.706
{0: [0, 2, 3, 5, 8], 1: [1, 4, 6, 7, 9]}
Total Happiness = 75.91
{0: [0, 1, 3, 7], 1: [2, 5], 2: [4, 6, 8, 9]}
Total Happiness = 63.166
{0: [0, 1, 3, 5], 1: [2, 6, 7], 3: [4, 8, 9]}
Total Happiness = 63.166
{0: [0, 1, 3, 5], 1: [4, 8, 9], 4: [2, 6, 7]}
Total Happiness = 54.625
{0: [0, 1, 7], 1: [4, 8, 9], 2: [2, 5], 4: [3, 6]}
Total Happiness = 54.62499999999999
{0: [0, 1, 7], 1: [4, 8, 9], 2: [2, 5], 6: [3, 6]}
Total Happiness = 49.160000000000004
{0: [0, 1, 7], 1: [4, 5, 8], 4: [3, 9], 7: [2, 6]}
Total Happiness = 42.228
{0: [0, 7], 1: [2, 4, 6], 3: [8, 9], 7: [1, 3, 5]}


In [38]:
graph, budget = read_input_file("phase1/test10.in")
d = convert_dictionary(gurobi_solver("phase1/test10.in", 2, 10))
is_valid_solution(dd, graph, budget, 2)

True